In [1]:
import matlab.engine
import numpy as np
import os
import json
import sys
import glob
from scipy import stats
from scipy.signal import argrelextrema
import datetime
import cv2
import time
import imutils
import png
import h5py
from scipy.io import loadmat
import smtplib

os.system("qmake --version")
os.system("python --version")

%matplotlib qt5
%matplotlib auto
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

sys.path.append('/home/gravishlab/Documents/Python/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/Tracker/')
from Tracker.Tracker import Tracker

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using matplotlib backend: Qt5Agg


In [8]:
%qtconsole

In [2]:
vid_locations = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'
file_list = []
file_list = glob.glob(os.path.join(vid_locations, '**/**/*-0000.mp4'))
# file_list = glob.glob(os.path.join(vid_locations, 'Tunnel_20180313-14/**/*0000.mp4'))
file_list = sorted(file_list)

print('Total Number of Videos: ',len(file_list))


#file_list = file_list[0:15] +file_list[278:292] +file_list[578:592] +file_list[878:902] #   [::20] #[0:3000:5]       
# file_list = file_list[0:500]
len(file_list)

Total Number of Videos:  8266


8266

In [3]:
trial_info = []

for file in file_list:
    tmps = file.split('/')
#     print(tmps[-1].split('_')[1])
#     print(os.path.splitext(tmps[-1])[0])
    trial_info.append({'substrate': tmps[-2],
                     'date': tmps[-1].split('_')[0],
                     'time': tmps[-1].split('_')[1],
                     'cam': tmps[-1].split('-')[0][-2:],
                     'colony': tmps[-3]})

### Define functions

In [4]:
# DEFINE FUNCTIONS

def track_video(file):
    print(kk, ' -- ', file)
    video = Tracker(file,min_object_size=40)
    
    if not video.file_exists_assoc:
        print('***', video.file_exists_assoc,  '*** -- ', file)
        
        
    if True:
        print('tracking')
        video.threshold_val = 0.2
        video.load_video()
        video.compute_background()          # form background image
        video.remove_background()           # remove background
        video.threshold()                   # threshold to segment features
        video.find_distance()               # remove dist before closing on clear bridge
        video.morpho_closing()
        video.find_objects()
#             print('-- deleted .h5 file for ant %i'%obj_idx)
        video.draw_contours()
        video.save_JSON()
        video.associate_contours(max_covariance=10,
                         max_velocity=100,
                         n_covariances_to_reject=20, 
                         max_tracked_objects=100,
                         kalman_state_cov=1,
                         kalman_init_cov=0.2,
                         kalman_measurement_cov=1)
        video.save_association_JSON()
        print('done')
        return video
    

def make_image_folder(file):
    basename = ('/').join(file.split('/')[:-1])
    foldername = file.split('/')[-1].split('.')[0]
    savelocation = ('/').join([basename, foldername])
    if not os.path.exists(savelocation):
        os.mkdir(savelocation)
        print('-- made new folder')
    else:
        print('-- folder already exists')
    return savelocation;


def run_SVM_for_LvR():
    SVM_path = '/media/gravishlab/SeagateExpansionDrive/AntTrack/LEAP_training/LvsR_trainingset/SVMdata.h5'
    from sklearn.grid_search import GridSearchCV
#     from sklearn.metrics import classification_report
#     from sklearn.metrics import confusion_matrix
    from sklearn.decomposition import PCA
    from sklearn.svm import SVC

    if os.path.exists(SVM_path):
        
        # LOAD IN FROM H5 APPROACH:
# #                 print('ant%i predictions exist'%ant_num)
#         hf = h5py.File(SVM_path,'r')
#         X = hf['X'][()].astype(np.float32) #hf.get('positions_pred')
#         y = hf['y'][()] #hf.get('conf_pred')
#         target_names = hf['target_names'][()]
#         target_names = target_names.astype('<U5')
    
# #         target_names_h5 = hf['target_names'][()]
# #         target_names_h5 = target_names_h5.astype('<U5')
#         labels = hf['labels'][()]
#         labels = labels.astype('<U5')
        
# #         target_names = np.array(list(set(labels))).astype('<U17')
# #         y = np.squeeze(np.array([np.where(np.isin(target_names,item))[0] for item in labels]))
# #         target_names = target_names.astype('<U5')
        
# #         print('h5 -- target names', target_names_h5)
#         print('labels -- target names' , target_names)
#         print(y[0:12])
#         print(labels[0:12])
#         plt.figure()
#         print(X[11].shape)
#         plt.imshow(np.reshape(X[11], (200,200)))
        
#         n_samples = X.shape[0]
#         n_classes = target_names.shape[0]
#         h = 200
#         w = 200
#         n_features = X.shape[1]
        
        
        # LOAD IN ALL IMAGES APPROACH ---- SLOW
        import imageio
        im_locations = '/media/gravishlab/SeagateExpansionDrive/AntTrack/LEAP_training/LvsR_trainingset'
        file_list = []
        file_list = glob.glob(os.path.join(im_locations, '**/**.png'))
        file_list = sorted(file_list)
        images = np.empty((0,200,200), int)
        labels = []#np.chararray((0,1))
        for im_path in file_list:
            image = imageio.imread(im_path)
            label = im_path.split('/')[-2]
            images = np.append(images, image[np.newaxis,:,:], axis = 0)
        #     print(images.shape, image[np.newaxis,:,:].shape)
        #     np.append(labels, [label], axis = 0)
            labels.append(label)
        labels = np.array(labels) 

        n_samples = images.shape[0]
        h = 200
        w = 200

        X = np.reshape(images, [n_samples, w*h])
        n_features = X.shape[1]
        target_names = np.array(list(set(labels))).astype('<U17')
        y = np.squeeze(np.array([np.where(np.isin(target_names,item))[0] for item in labels]))
        n_classes = target_names.shape[0]

        
        # compute eigenimages (unsupervised feature extraction)
        n_components = 50
        pca = PCA(n_components = n_components, svd_solver = 'randomized', whiten = True).fit(X)
        # Projecting the input dat on the eigenimage orthonormal basis 
        x_pca = pca.transform(X)
        # Train SVM classification model
        param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
             'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
        clf = GridSearchCV(SVC(kernel ='rbf', class_weight = 'balanced'), param_grid)
        clf = clf.fit(x_pca, y)
        print('--- best estimator found by grid search:')
        print('---', clf.best_estimator_)
    else:
        print('--- No data for SVM')
    
    return pca, clf, target_names





    
def rotate_and_zoom_frame(video, obj_OI, fr_num, buffer, x_dim, y_dim):
    idc = np.where(obj_OI['frames'] == fr_num)[0][0]
    x = obj_OI['x'][idc]
    y = obj_OI['y'][idc]

    # set buffer around bkgd div image so ant is always in center for zoomed-in view
    temp = video.frames_normed[int(fr_num)]
    blank_frame = np.ones((y_dim+ 2*buffer, x_dim+ 2*buffer))* 0.7 #1.001# np.max(temp) # gray background  1.0001#
#     blank_frame = np.zeros((y_dim+ 2*buffer, x_dim+ 2*buffer)) # black background
    frame = blank_frame.copy()
    frame[buffer:-buffer, buffer:-buffer] = temp

    # zoom into around ant
    xrange = range(int(round(x)), int(round(x+2*buffer)))
    yrange = range(int(round(y)), int(round(y+2*buffer)))
    xrange_actual = sorted(list( set(xrange) & set(range(0, x_dim+2*buffer) ) ))
    yrange_actual = sorted(list( set(yrange) & set(range(0, y_dim+2*buffer) ) ))
    frame_zoom = frame[np.ix_(yrange_actual, xrange_actual)]

    # rotate ant
    rotation = obj_OI['angle_smooth'][idc]
    if not np.isnan(rotation):
        rotated = imutils.rotate_bound(frame_zoom, -1*rotation)
        rot_ranges = [range(int(x/2-100), int(x/2+100)) for x in rotated.shape];
        rotated_zoom = rotated[np.ix_(rot_ranges[0], rot_ranges[1])]
    else:
        rotated_zoom = np.zeros((200,200))
        
    return frame, frame_zoom, rotated_zoom;


def adjust_gain(temp, bkgd_cutoff = 5):                
    # adjust gain of image -- option 2
    temp = -1*(temp-1)
    temp = temp/np.max(temp)
    cutoff = 5
    temp =(255-bkgd_cutoff)*temp + bkgd_cutoff
    temp[temp<0]=0
    
    # adjust gain of image -- option 1
#     print(np.min(temp),np.max(temp))
#     temp = 2-temp
#     min_o = np.min(temp)
#     max_o = np.max(temp)
#     temp = ((temp - min_o)/(max_o-min_o)*255)
#     cutoff = (1-min_o)*1.4*255
#     print(cutoff)
#     temp[temp<bkgd_cutoff]=temp[temp<bkgd_cutoff]-10
#     temp[temp<0]=0
    return temp

# plot full back div frame, zoomed in on ant, and rotated zoomed-in view
def plot_compilation_fig_fn(frame, frame_zoom, rotated_zoom, buffer):
    plt.close('all')
    fig = plt.figure()
    gs = gridspec.GridSpec(2,2)
    gs.tight_layout(fig, rect = [0.5, 0, 1,1], h_pad = 0.5)
    ax1 = plt.subplot(gs[0,0:2])
    ax2 = plt.subplot(gs[1,0])
    ax3 = plt.subplot(gs[1,1])

    ax1.clear()
    ax1.imshow(frame[buffer:-buffer, buffer:-buffer])
#     ax1.plot(x,y,'.r', markersize =2)
#     ax1.invert_yaxis()

    ax2.imshow(frame_zoom)
#     ax2.invert_yaxis()
    ax2.axis('off')

    ax3.clear()
    ax3.axis('off')
    if not np.max(rotated_zoom)==0:
        ax3.imshow(rotated_zoom)
#                 ax3.imshow(rotated)
#         ax3.invert_yaxis()
    plt.pause(0.2)
    return;


# plot only zoomed-in and rotated image
def plot_rotated_img_fn(rotated_zoom, fr_num):
    if not np.max(rotated_zoom)==0:
        plt.close('all')
        plt.figure(figsize = (5,5))
        plt.imshow(rotated_zoom, cmap = 'gray')
        plt.text(10,10,'Fr: %i'%fr_num, color = 'w')
        plt.gca().axis('off')
        plt.pause(0.5)
    return;

def save_video(folder, base):
    # save images as movie
    if os.path.isfile((folder + '/' + base + ".mp4")):
        os.remove(folder + '/' + base + ".mp4")
        print('** Deleted .mp4 file')
    print('saving .mp4 file')
#     command_to_run = "ffmpeg -r 20 -i '" + folder  + "/" + base + "_fr%03.0.png' -vcodec libx264 '" + folder + "/" + base + ".mp4'"
#     print(command_to_run)
    os.system("ffmpeg -pattern_type glob -r 20 -i '" + folder  + "/" + base + "_fr*.png' -vcodec libx264 '" 
              + folder + "/" + base + ".mp4'")
    plt.pause(4)
    
    # delete all trackway vids
    pics2delete = glob.glob(os.path.join(folder, base+'_fr*.png'))
    for pic in pics2delete:
        os.remove(pic)
    return
    
    
# i created a gmail account just for this purpose
username = 'codefromglenna'
password = 'imacomputer'
fromaddr = 'codefromglenna@gmail.com'
toaddr   = '6106392662@vtext.com'

def sendtxt(msg):
    server = smtplib.SMTP('smtp.gmail.com:587');
#     server.connect();
    server.starttls();
    server.login(username,password);
    server.sendmail(fromaddr, toaddr, msg) # -- ' + time.ctime() )
    server.quit()

### Define function to find flipped angles, smooth, and interpolate

In [5]:
# FILP ANGLE FUNCTIONS

# for a given frame, determine if angle is flipped
# def find_flipped_angles(obj, fr_OI, plots_on = False):
#     all_angs = obj_OI['angle_improved']
#     all_frs = obj_OI['frames']
#     fr_OI_idc = np.where(all_frs == fr_OI)[0][-1]
    
#     # just look at window around frame
#     win_idcs = np.logical_and( np.logical_and(all_frs >= fr_OI - 10, all_frs <= fr_OI + 10), all_frs != fr_OI) 
#     win_frs = all_frs[win_idcs]
#     win_angs = all_angs[win_idcs]

#     if plots_on:
#         plt.plot(all_frs, np.sin(np.deg2rad(all_angs)), '.k', alpha = 0.2)
#         plt.plot(all_frs, np.cos(np.deg2rad(all_angs)), '.b', alpha = 0.2)
#         plt.plot(win_frs, np.sin(np.deg2rad(win_angs)), '.k')
#         plt.plot(win_frs, np.cos(np.deg2rad(win_angs)), '.b')
#         plt.plot(fr_OI, np.sin(np.deg2rad(all_angs[fr_OI_idc])), '*r')
#         plt.plot(fr_OI, np.cos(np.deg2rad(all_angs[fr_OI_idc])), '*r')

#     # what are sin and cos of nearby frames
#     win_sin = np.nanmedian(np.sin(np.deg2rad(win_angs)))
#     win_cos = np.nanmedian(np.cos(np.deg2rad(win_angs)))
#     fr_sin = np.sin(np.deg2rad(all_angs[fr_OI_idc]))
#     fr_cos = np.cos(np.deg2rad(all_angs[fr_OI_idc]))
#     is_flipped = np.logical_or(np.abs(fr_sin - win_sin) > 0.5, np.abs(fr_cos - win_cos) > 0.5)
#     print(is_flippe[-30:])
    
#     # smooth out angle by av
# #     if is_flipped:
# #         print('Fr: %0.0f -- sin: %0.2f vs. %0.2f -- cos: %0.2f vs. %0.2f'%(fr_OI, fr_sin, win_sin, fr_cos, win_cos))
#     return is_flipped, all_angs[fr_OI_idc]


def calc_mov_avg(x, N, cutoff_nan):
    padded_x = np.insert(np.insert( np.insert(x, len(x), np.empty(int(N/2))*np.nan), 0, np.empty(int(N/2))*np.nan ),0,0)
    n_nan = np.cumsum(np.isnan(padded_x)) 
    cumsum = np.nancumsum(padded_x) 
    window_sum = cumsum[N+1:] - cumsum[:-(N+1)] - x
    window_n_nan = n_nan[N+1:] - n_nan[:-(N+1)] - np.isnan(x)
    window_n_values = (N - window_n_nan).astype(float)
    window_n_values[window_n_values<cutoff_nan] = np.nan # if fewer than cutoff values in window, ignore
    movavg = (window_sum) / (window_n_values)
    return movavg


def find_flipped_angles_all(obj, plots_on = False):
    
    all_angs = obj_OI['angle_improved']
    all_frs = obj_OI['frames']
    all_sin = np.sin(np.deg2rad(all_angs))
    all_cos = np.cos(np.deg2rad(all_angs))
    
    if plots_on:
        plt.figure()
        plt.subplot(3,1,1)
        plt.plot(all_frs, all_sin, '-k', alpha = 0.3)
        plt.plot(all_frs, all_sin, '.g', alpha = 0.3)
        plt.plot(all_frs, all_cos, '-k', alpha = 0.3)
        plt.plot(all_frs, all_cos, '.b', alpha = 0.3)

    # FLIP EVERY OTHER SECTION DEFINED WHERE COS OR SIN CHANGES BY MORE THAN 1
    idcs = np.where(np.logical_not(np.isnan(all_sin)))[0]
    d_sin = np.abs(np.diff(all_sin[idcs]))
    d_cos = np.abs(np.diff(all_cos[idcs]))
    d_big = np.logical_or(d_sin> 1.5, d_cos > 1.5)
    d_big_cumsum = np.cumsum(np.insert(d_big,0,0))
    d_big_opp = (d_big_cumsum%2).astype(bool)
    
    if plots_on:
        plt.plot(all_frs[idcs[d_big_opp]], all_sin[idcs[d_big_opp]], '.r', MarkerSize = 2)
        plt.plot(all_frs[idcs[d_big_opp]], all_cos[idcs[d_big_opp]], '.r', MarkerSize = 2)
    
    all_sin[idcs[d_big_opp]] = -1* all_sin[idcs[d_big_opp]]
    all_cos[idcs[d_big_opp]] = -1* all_cos[idcs[d_big_opp]]
    
    def remove_90_turns(arr, all_frs):
        idcs = np.where(np.isfinite(arr))[0]
        d_arr = np.diff(arr[idcs])
        d_sign = np.sign(d_arr)
        d_med = np.logical_and( np.abs(d_arr) > 0.5, np.abs(d_arr) < 1.5)
        d_med_cumsum = np.cumsum(np.insert(d_med,0,0))
        d_med_opp = np.cumsum(np.insert(d_sign*d_med,0,0))#(d_med_cumsum%2).astype(bool)
        if np.sum(d_med_opp==0)<np.sum(d_med_opp != 0):
            d_med_opp = d_med_opp - np.median(d_med_opp)
        arr[idcs]=np.clip(arr[idcs]-d_med_opp, -1, 1)  
        return arr
    all_sin = remove_90_turns(all_sin, all_frs)
    all_cos = remove_90_turns(all_cos, all_frs)

    # FIND WHERE DIFFERS FROM MOVING AVE BY AMOUNT AND SWITCH
#     for reps in range(0,2):
#         print(reps)
#         sin_flip_comparison = calc_mov_avg(all_sin, 20, 1)
#         cos_flip_comparison = calc_mov_avg(all_cos, 20, 1)

#         is_flipped = np.logical_or(np.abs(all_sin - sin_flip_comparison) > 1, np.abs(all_cos - cos_flip_comparison) > 1)
#         all_sin[is_flipped] = -1* all_sin[is_flipped]
#         all_cos[is_flipped] = -1* all_cos[is_flipped]
#         if plots_on:
#             plt.subplot(2,1,1)
#             plt.plot(all_frs, all_sin, '-k', alpha = 0.3)
#             plt.plot(all_frs, all_sin, '.k', alpha = 0.3)
#             plt.plot(all_frs, calc_mov_avg(all_sin, 20, 1), '.b', alpha = 0.3)
#             plt.plot(all_frs, all_cos, '-k', alpha = 0.3)
#             plt.plot(all_frs, all_cos, '.k', alpha = 0.3)
#             plt.plot(all_frs, calc_mov_avg(all_cos, 20, 1), '.g', alpha = 0.3)
        
    all_sin_good = all_sin.copy()
    all_cos_good = all_cos.copy()
    all_sin_smooth = calc_mov_avg(all_sin_good, 10, 2)
    all_cos_smooth = calc_mov_avg(all_cos_good, 10, 2)
    all_angs_good = np.rad2deg(np.arctan2(all_sin_smooth, all_cos_smooth))

    if plots_on:
        plt.subplot(3,1,2)
        plt.plot(all_frs, all_sin_good, '-k', alpha = 0.3)
        plt.plot(all_frs, all_sin_good, '.k', alpha = 0.3)
        plt.plot(all_frs, all_cos_good, '-k', alpha = 0.3)
        plt.plot(all_frs, all_cos_good, '.k', alpha = 0.3)
        plt.plot(all_frs, all_sin_smooth, '.g', alpha = 0.3)
        plt.plot(all_frs, all_cos_smooth, '.b', alpha = 0.3)
        plt.subplot(3,1,3)
        plt.plot(all_frs, all_angs, 'k', alpha = 0.3)
        plt.plot(all_frs, all_angs_good, 'r')
    return all_angs_good


# # identify and flip any frames that are clearly facing towards the tail
# def change_flipped_angles(obj, N, M, reps, plots_on = False):
#     frs = obj['frames']
#     raw_angs = obj['angle_improved']
#     if np.sum(np.logical_not(np.isnan(raw_angs))) < 20:
#         raw_angs = obj['angle']
#         print(' -- not enough angle improved values, using contour angles')
#     raw_angs_shift = (raw_angs+ 180)%360  
#     angs = (raw_angs+180)%360 
    
#     for kk in range(0, reps):
#         d_angs = np.diff(angs)
#         d_angs_big = np.logical_and(d_angs>175, d_angs<185)
#         nonan = np.logical_not(np.isnan(angs))
#         angs_nonan = angs[nonan]
#         frs_nonan = frs[nonan]
#         # buffer edges with median of angle across whole trial
#         angs_buff = np.insert(np.insert(angs_nonan, angs_nonan.size, np.ones(int(N/2+1))*np.nanmedian(angs)), 0, np.ones(int(N/2+1))*np.nanmedian(angs))
#         # make mask so ignore nans in moving average
#         mask = np.isnan(angs_buff)
#         K = np.ones(int(N+M+1), dtype=int)
#         K[int(N/2):int(N/2+M+1)]=0
#         out = np.convolve(np.where(mask, 0, angs_buff), K, mode = 'same')/np.convolve(~mask,K, mode = 'same')
#         wrong_way = np.logical_and(np.abs(angs_nonan-out[int(N/2+1):-int(N/2+1)])>150, np.abs(angs_nonan-out[int(N/2+1):-int(N/2+1)])<200)
#         angs_nonan[wrong_way] = angs_nonan[wrong_way] 
#         new_angs = angs.copy()
#         new_angs[nonan]=angs_nonan
#         angs = new_angs
# #     new_angs = new_angs-180
        
#     if plots_on:
#         plt.figure()
#         plt.subplot(2,1,1)
#         plt.plot(frs, raw_angs, '.k')
#         plt.subplot(2,1,2)
#         plt.plot(frs, raw_angs_shift, '.k')
#         plt.plot(frs, raw_angs_shift, '-k')
#         plt.plot(frs, new_angs, '.r', MarkerSize =2)
    
#     return new_angs-180


# average x,y,angle if isolated frame without head found (angle_improve = nan)
def find_nan_gaps(arr, limit):  
    from itertools import groupby
    yy = np.isnan(arr)
    xx = range(len(yy))
    where_gapOI = np.full(arr.shape, False)
    where_othergaps = np.full(arr.shape, False)
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == True: # if is a group of nan
            g = list(g)
            if any(x in g for x in [0, len(arr)-1]): # if first or last group
                where_othergaps[np.array(g)]=True
                continue       
            if len(g)<= limit: # length is below limit
                where_gapOI[np.array(g)]=True
    return where_gapOI
def find_interp_idcs(where_interpolate):
    interp_idcs = []
    for val in [-1,0,1]:
        interp_idcs = np.concatenate([interp_idcs,np.where(where_interpolate)[0]+val])
    interp_idcs = np.sort(np.array(list(set(interp_idcs)))) # get of repeat elements
    interp_idcs = interp_idcs[np.logical_and(interp_idcs>-1, interp_idcs < len(where_interpolate))].astype(np.uint32) # only elements in range
    return interp_idcs
def interp_vals(arr, interp_idcs): # array includes nan values
    temp = arr[interp_idcs]
    interpolated_vals = np.interp(
        interp_idcs, 
        interp_idcs[np.logical_not(np.isnan(temp))], temp[np.logical_not(np.isnan(temp))] )
    interp = arr.copy()
    interp[interp_idcs] = interpolated_vals
    return interp
# def averagexyangle(obj): # old version
#     where_isolated_nan = np.where( np.concatenate(([False], 
#                                          np.diff(np.isnan(obj['angle_improved']).astype(np.float64) , 2)== -2, 
#                                          [False] )  ))[0]
#     for idc in where_isolated_nan:
#         entries_to_ave = [idc-1, idc+1]
#         obj['x'][idc] = np.mean(obj['x'][entries_to_ave])
#         obj['y'][idc] = np.mean(obj['y'][entries_to_ave])
#         obj['angle_improved'][idc] = np.mean(obj['angle_improved'][entries_to_ave])
#     return  obj
    

### Run through trials - MAIN BODY

In [25]:
plt.close('all')

track_vid = 1
plot_compilation_fig = 0
plot_rotated_img = 0
plot_angle_fixes = False
save_images = 0
save_videos = 0
save_hdf5 = 1
run_model_predictions = 1

# run SVM to determine L or R of each image
print('Running PCA and making SVM model')
pca, clf, target_names = run_SVM_for_LvR()
print('\n')

starting_file = 0 # antennae files: 0, 400, 700, 900
for kk, file in enumerate(file_list[starting_file:(starting_file+5)]):
#     print(file, '***')

    if kk%250 == 0:
        sendtxt('analyzed %i videos'%kk)
    
    # track video to get up-to-date head finding
    if track_vid:
        video = track_video(file)
    elif 'video' in globals():
        if (video.videoname != file):
            video = track_video(file)
    else:
        video = track_video(file)

    # make folder for images if not already present
    savelocation = make_image_folder(file)

    # find video dimensions
    x_dim = video.frames_normed[0].shape[1]
    y_dim = video.frames_normed[0].shape[0]

    # for each tracked ant
    for obj_idx in range(0,len(video.objects)):
#         print('Object %i'%obj_idx)
        
        allframes = []
        
        # limit to tracked ant data
        obj_OI = video.objects[obj_idx].copy()
        
        # if only short fragment of trackway, don't track
        if len(obj_OI['frames'])<50:
            print('not enough tracked frames. skipping ant %i'%obj_idx)
            continue
           
        # if ant is close to edges for most of trial, don't track
        n_away_from_edge = np.sum(np.logical_not(np.any( # how many points aren't close to edge
            np.abs([obj_OI['x']-video.Width/2,obj_OI['y']-video.Height/2]) > 
            np.array([video.Width/2 - 50,video.Height/2 - 50])[:,np.newaxis],axis=0) ))
        if n_away_from_edge < 30:
#             print('-- %i points far from edge. skipping ant %i'%(n_away_from_edge,obj_idx))
            continue
#         else:
#             print('-- ant%i has %i points away from edge'%(obj_idx, n_away_from_edge))
        
    
        # CORRECT FLIPPED ANGLES
        obj_OI['angle_smooth'] =  find_flipped_angles_all(obj_OI, plots_on = plot_angle_fixes)
#         obj_OI['angle_improved'] = change_flipped_angles(obj_OI, 20, 10, 4, plots_on = False) # old method
        # average x,y,angle for gaps of <N nan
        where_interp =find_nan_gaps(obj_OI['angle_improved'], 4)
        if np.sum(where_interp)>0:
            obj_OI['angle_smooth'] = interp_vals(obj_OI['angle_smooth'], 
                                                   find_interp_idcs(where_interp))
        video.objects[obj_idx]['angle_improved']=obj_OI['angle_smooth'].copy()

        

        # rotate each frame and plot or save
        for f_id, fr_num in enumerate(obj_OI['frames']):
            
            # if ant center is too close to a wall, make it an empty frame
            if np.any( np.abs([obj_OI['x'][f_id]-video.Width/2, obj_OI['y'][f_id]-video.Height/2]) > 
                np.array([video.Width/2 - 50, video.Height/2 - 50]) ):
                clean_frame = np.ones((200,200),dtype=np.uint8)
#                 video.objects[obj_idx]['angle_improved'][f_id] = np.nan
                
            else:

                buffer = 150 # how much pad around frame so that rotating zoomed in view keeps ant in center of frame?
                frame, frame_zoom, rotated_zoom = rotate_and_zoom_frame(video, obj_OI, fr_num, buffer, x_dim, y_dim)

                # adjust gain so that background is must darker than subject
                gain_frame = adjust_gain(rotated_zoom, bkgd_cutoff = 5)

                # get rid of contours
                glenna= gain_frame.copy()
                glenna[gain_frame>25]=1
                glenna[gain_frame<=25]=0
                kernel = cv2.getStructuringElement( cv2.MORPH_CROSS, (3,3))
                glenna_close = cv2.morphologyEx(255*glenna.astype('uint8'), cv2.MORPH_CLOSE, kernel, iterations = 1)
                _,contours,_ = cv2.findContours(glenna_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
                small_contours = [c for c in contours if cv2.contourArea(c) < 10]
                large_contours = [c for c in contours if cv2.contourArea(c) > 30]
                edge_contours = [c for c in contours if 
                                 np.any(np.logical_or(c == 1, c == 199))]
#                 ant_contours = [c for c in large_contours if not
#                                  np.any(np.logical_or(c == 1, c == 199))]

                # remove contours touching the edge
                clean_frame = gain_frame.copy()
                clean_frame = cv2.drawContours(clean_frame, small_contours, -1, 3, -1)
                clean_frame = cv2.drawContours(clean_frame, edge_contours, -1, 3, -1)

                # if removed all ant-sized contours, make it a blank frame
                if np.mean(clean_frame[95:105, 95:105]) < 100 :
    #                 print('fr: %i - all ant blobs removed, %0.1f'%(fr_num, np.mean(clean_frame[95:105, 95:105])))
                    clean_frame = np.ones((200,200),dtype=np.uint8)
#                     video.objects[obj_idx]['angle_improved'][f_id] = np.nan

#                 else: # if not blank, correct any flipped frames
#                     # CORRECT ABBERANT ANGLES USING SVM
#                     im_pca = pca.transform( np.reshape(clean_frame, [1, 200*200]))
#                     im_pred = clf.predict(im_pca)

#                     if target_names[im_pred] == 'Left' :
#                         confs = clf.decision_function(im_pca)[0]
#                         print('--- SVM thinks frame %0.0f is flipped. conf = %0.2f, %0.2f, %0.2f'%(fr_num, confs[0], confs[1], confs[2] ))
#                         plt.figure()
#                         plt.imshow(clean_frame)
#                         apiojawg

    #     #                 print(target_names[im_pred])
    #     #                 plt.imshow(clean_frame)
    #     #                 plt.figure()
    #     #                 plt.imshow(np.fliplr(np.flipud(clean_frame)))
    # #                     print('---- SVM thinks frame %0.0f is flipped, ang %0.1f'%(fr_num, fr_ang))
    #                     if is_flipped:
    # #                         print('---- comparison of sin/cos thinks frame %0.0f is flipped'%fr_num)
    #                         clean_frame = np.fliplr(np.flipud(clean_frame))
    #                         video.objects[obj_idx]['angle_improved'][f_id] = -1*np.sign(fr_ang)*(180-np.abs(fr_ang))
    #                         print('----- switched frame %0.0f'%fr_num)

                
            allframes.append(clean_frame)
                
            if plot_compilation_fig:
                plot_compilation_fig_fn(frame, frame_zoom, rotated_zoom, buffer)

            if plot_rotated_img:
                plot_rotated_img_fn(clean_frame, fr_num)
                

        # check if all video is backwards:
        image_classifications = np.squeeze([clf.predict(pca.transform( np.reshape(im, [1, 200*200]))) for im in allframes[0:100]])
        image_classifications = np.insert(image_classifications,0,[0,1,2]) # make sure has at least one 0, 1, and 2
        classification_counts = np.bincount(image_classifications)
#         print('R: ', classification_counts[target_names=="Right"], ' L: ', classification_counts[target_names=="Left"])
        if classification_counts[target_names=="Right"] < classification_counts[target_names=="Left"]:
            allframes = [np.fliplr(np.flipud(fr)) for fr in allframes]
            video.objects[obj_idx]['angle_improved'] = video.objects[obj_idx]['angle_improved'] + 180
            print('--- ant facing wrong way in video. flipping.')
            
        if save_images:
            for ff,frame in enumerate(allframes):
#                 if not np.max(rotated_zoom)==0:
                fr_to_save = frame.astype(np.uint8) #((temp/np.max(temp)*255)-255).astype(np.uint8) #np.abs(((temp)/np.max(temp)*255)-255)
                png.from_array(fr_to_save, 'L').save("%s/ant%i_fr%i.png"%(savelocation, obj_idx, ff))

            # save transparent ant cutouts for figure
#             if obj_idx == 2:
#                 frame_idcs = list(np.where(np.isin(video.objects[obj_idx]['frames'], [193,431]))[0])
#                 print(frame_idcs)
#                 for ff in frame_idcs: # save transparent cut-outs of ants for paper figure
#                     frame = 255-allframes[ff]
#                     rgb_frame = cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_GRAY2RGB)
#                     rgba_frame = np.append(rgb_frame, 255*np.zeros((200,200))[:,:,np.newaxis].astype(np.uint8), axis=2).astype(np.uint8)
#                     t_idcs=rgba_frame[:,:,0]>230
#                     t_idcs_open = cv2.morphologyEx(t_idcs.astype(np.uint8), cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
#                     _,contours,_= cv2.findContours(t_idcs_open, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#                     tmp = np.zeros((200,200))
#                     cv2.drawContours(tmp, contours, np.argmax(np.array([len(c) for c in contours])), 1, -1)
#                     rgba_frame[:,:,3]=tmp*255
#                     plt.figure()
#                     plt.imshow(rgba_frame, cmap = 'gray')
        
        if save_videos:
            save_video(savelocation, 'ant%i'%obj_idx)


        # save as a hdf5 file
        if save_hdf5:
            temp =np.array([allframes])
            temp = np.swapaxes(temp,0,1)
            temp = np.swapaxes(temp,2,3)
            hf = h5py.File("%s/ant%i.h5"%(savelocation, obj_idx),'w')
            hf.create_dataset('box', data=temp)
            hf.close()
            print('-- Saved .h5 file for ant %i'%obj_idx)
#             video.save_JSON()
#             video.associate_contours(max_covariance=10,
#                              max_velocity=100,
#                              n_covariances_to_reject=20, 
#                              max_tracked_objects=100,
#                              kalman_state_cov=1,
#                              kalman_init_cov=0.2,
#                              kalman_measurement_cov=1)
            video.save_association_JSON()
#             print('-- re-saved contours and associated trackways for improved angle detection')
            
        if run_model_predictions:
            start_time = time.time()
            pfunc_path = '/home/gravishlab/Documents/MATLAB/leap/leap/predict_box.py'
            h5_path = "%s/ant%i.h5"%(savelocation, obj_idx)
#             model_path = '/home/gravishlab/Documents/MATLAB/leap/models/fast_train/180614_094549-n=209/final_model.h5'
            model_path = '/media/gravishlab/SeagateExpansionDrive/AntTrack/LEAP_training/models/180808_175924-n=679/final_model.h5'
            save_path = "%s/ant%i_predictions.h5"%(savelocation, obj_idx)
        
#             # FOR ANTENNAE TRACKING
#             model_path = '/media/gravishlab/SeagateExpansionDrive/AntTrack/LEAP_training/models/180727_114754-n=428/final_model.h5'
#             save_path = "%s/ant%i_antennae_predictions.h5"%(savelocation, obj_idx)
        
            command = "python3 \'%s\' \'%s\' \'%s\' \'%s\' "%(pfunc_path, h5_path, model_path, save_path)
            
            if os.path.exists(save_path):
                os.remove(save_path)
#             print(command)
            os.system(command)
            print('-- Saved model predictions for ant %i -- %0.1f s'%(obj_idx, time.time()-start_time))
        
            # remove .h5 because they're large
            os.remove(h5_path)
            print('-- deleted .h5 file for ant %i'%obj_idx)


#     del video
        
print('\n\nALL DONE!!!\n')

0  --  /media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/1mm/20180314_084502_16276712-0000.mp4
tracking
done
-- folder already exists
-- Saved .h5 file for ant 0
-- Saved model predictions for ant 0 -- 10.2 s
-- deleted .h5 file for ant 0
-- Saved .h5 file for ant 1
-- Saved model predictions for ant 1 -- 5.0 s
-- deleted .h5 file for ant 1
1  --  /media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/1mm/20180314_084620_16276712-0000.mp4
tracking
done
-- folder already exists
-- Saved .h5 file for ant 0
-- Saved model predictions for ant 0 -- 4.9 s
-- deleted .h5 file for ant 0
-- Saved .h5 file for ant 1
-- Saved model predictions for ant 1 -- 4.6 s
-- deleted .h5 file for ant 1
-- Saved .h5 file for ant 2
-- Saved model predictions for ant 2 -- 4.5 s
-- deleted .h5 file for ant 2
-- Saved .h5 file for ant 3
-- Saved model predictions for ant 3 -- 4.8 s
-- deleted .h5 file for ant 3
not enough tracked frames. skipping ant 4
-- Saved .h5 file for ant 5
-

## Just run predictions on all files 

In [7]:
for kk, file in enumerate(file_list[0:10]):    
    # make folder for images if not already present
    basename = ('/').join(file.split('/')[:-1])
    foldername = file.split('/')[-1].split('.')[0]
    savelocation = ('/').join([basename, foldername])
    print(savelocation)
    if not os.path.exists(savelocation):
        print('-- no folder! skipping... ')
        continue
      
    
    # find all ant files:
    afiles = glob.glob(os.path.join(savelocation, 'ant*[0-9].h5'))
#     print(afiles)
    
    start_time = time.time()
    pfunc_path = '/home/gravishlab/Documents/MATLAB/leap/leap/predict_box.py'
    model_path = '/media/gravishlab/SeagateExpansionDrive/AntTrack/LEAP_training/models/180802_131333-n=639/final_model.h5'

    for h5_path in afiles:
#     h5_path = "%s/ant%i.h5"%(savelocation, obj_idx)
# #             model_path = '/home/gravishlab/Documents/MATLAB/leap/models/fast_train/180614_094549-n=209/final_model.h5'
        ant_num = h5_path.split('ant')[-1].split('.')[0]
        save_path = "%s/ant%s_predictions.h5"%(savelocation, ant_num)
        command = "python3 \'%s\' \'%s\' \'%s\' \'%s\' "%(pfunc_path, h5_path, model_path, save_path)

        if os.path.exists(save_path):
            os.remove(save_path)
    #             print(command)
        os.system(command)
        print('-- Saved model predictions for ant %s -- %0.1f s'%(ant_num, time.time()-start_time))
        
print('\n\nALL DONE!!!')

/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080112_16276736-0000
-- Saved model predictions for ant 0 -- 5.0 s
-- Saved model predictions for ant 1 -- 11.0 s
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080315_16276736-0000
-- Saved model predictions for ant 6 -- 5.4 s
-- Saved model predictions for ant 7 -- 10.4 s
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080418_16276736-0000
-- Saved model predictions for ant 0 -- 5.5 s
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080521_16276736-0000
-- Saved model predictions for ant 0 -- 5.3 s
-- Saved model predictions for ant 1 -- 10.0 s
-- Saved model predictions for ant 11 -- 14.2 s
-- Saved model predictions for ant 12 -- 18.1 s
-- Saved model predictions for ant 15 -- 22.5 s
-- Saved model predictions for ant 16 -- 27.0 s
-- Saved model predictions for ant 2 -- 31.4 s
/media/gravishlab/SeagateExp

In [33]:
plt.close('all')
frs = video.objects[14]['frames']
# plt.figure()
# plt.plot(frs, video.objects[0]['angle_improved'],'k.')
angs =video.objects[14]['angle_improved']%360 +180
d_angs = np.diff(angs)
d_angs_big = np.logical_and(d_angs>175, d_angs<185)
plt.figure()
plt.plot(frs,angs,'.b')
plt.plot(frs,angs,'-b')

plt.plot(frs[:-1][d_angs_big], angs[:-1][d_angs_big],'.r')

N = 20
M = 8
reps = 4
# key = np.ones((N+3,))/N
# key[int(N/2-1):int(N/2+2)]=0
# mask = np.isnan(angs)

def remove_the_stuff(arr):
    plt.figure()
    plt.plot(frs,angs,'.b')
    plt.plot(frs,angs,'-b')

    nonan = np.logical_not(np.isnan(angs))
    angs_nonan = angs[nonan]
    frs_nonan = frs[nonan]
    angs_buff = np.insert(np.insert(angs_nonan, angs_nonan.size, np.ones(int(N/2+1))*np.nanmedian(angs)), 0, np.ones(int(N/2+1))*np.nanmedian(angs))
    # mov_ave = np.convolve(angs_buff, key, mode='valid')
    # plt.plot(frs, mov_ave, ':g')

    mask = np.isnan(angs_buff)
    K = np.ones(int(N+M+1), dtype=int)
    K[int(N/2):int(N/2+M+1)]=0
    out = np.convolve(np.where(mask, 0, angs_buff), K, mode = 'same')/np.convolve(~mask,K, mode = 'same')
    plt.plot(frs[np.logical_not(np.isnan(angs))], out[int(N/2+1):-int(N/2+1)], ':k')

    plt.plot(frs[np.logical_not(np.isnan(angs))],np.abs(angs_nonan-out[int(N/2+1):-int(N/2+1)]),'-r')
    wrong_way = np.logical_and(np.abs(angs_nonan-out[int(N/2+1):-int(N/2+1)])>150, np.abs(angs_nonan-out[int(N/2+1):-int(N/2+1)])<200)
    print(frs_nonan[wrong_way])
    angs_nonan[wrong_way] = (angs_nonan[wrong_way] - 180)%360
    plt.plot(frs_nonan[wrong_way], angs_nonan[wrong_way], '.g')

    new_angs = angs.copy()
    new_angs[nonan]=angs_nonan
    return new_angs

for kk in range(0, reps):
    print(kk)
    angs = remove_the_stuff(angs)
# plt.figure()
# plt.plot(frs, new_angs, '.b')


# N = 10 # how many points to look around (not including point of interest and direct neighbors)
# nonan = np.logical_not(np.isnan(angs))
# angs_nonan = angs[nonan]
# frs_nonan = frs[nonan]
# # buffer edges with median of angle across whole trial
# angs_buff = np.insert(np.insert(angs_nonan, angs_nonan.size, np.ones(int(N/2+1))*np.nanmean(angs)), 0, np.ones(int(N/2+1))*np.nanmean(angs))
# # make mask so ignore nans in moving average
# mask = np.isnan(angs_buff)
# K = np.ones(int(N+3), dtype=int)
# K[int(N/2):int(N/2+3)]=0 # ignore pointOI and its neighbors
# out = np.convolve(np.where(mask, 0, angs_buff), K, mode = 'same')/np.convolve(~mask,K, mode = 'same')
# wrong_way = np.logical_and(np.abs(angs_nonan-out[int(N/2+1):-int(N/2+1)])>160, np.abs(angs_nonan-out[int(N/2+1):-int(N/2+1)])<200)
# angs_nonan[wrong_way] = (angs_nonan[wrong_way] - 180)%360
# new_angs = angs.copy()
# new_angs[nonan]=angs_nonan
# plt.figure()
# plt.plot(frs, angs, '.k')
# plt.plot(frs, new_angs, '.r', MarkerSize = 2)




0
[]
1
[]
2
[]
3
[]


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in less
  import sys


# Load h5 file and get info

In [76]:
import h5py

hf = h5py.File('/home/gravishlab/Downloads/2018-05-03_cluster-sampled.k=10,n=150.h5','r')
glenna = hf.get('box')
print(glenna)
print(glenna[0].shape)

framesIDx = hf.get('framesIdx')
print(framesIDx)
print(framesIDx[0][0:10])

<HDF5 dataset "box": shape (1500, 1, 192, 192), type "|u1">
(1, 192, 192)
<HDF5 dataset "framesIdx": shape (1, 1500), type "<f8">
[210103. 162451. 329233. 122019. 107701. 158369. 337175. 231041. 226709.
 210825.]


In [158]:
hf = h5py.File('/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080112_16276736-0000/ant1.h5','r')
# hf = h5py.File('/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080112_16276736-0000/training/ant0_fasttrain.h5','r')
print(hf.keys())
glenna = hf.get('box')
print(glenna)
glenna[0].shape




KeysView(<HDF5 file "ant1.h5" (mode r)>)
<HDF5 dataset "box": shape (589, 1, 200, 200), type "<f8">


(1, 200, 200)

# Combine .h5 files to make larger training set - KEEP ALL SKELETON POINTS

In [31]:
# import tracked h5s to combine
base_folder = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'
# FILES FOR 180621_133300 --> FullTrainingSet_v1
# tracked_files = [ \
#                   base_folder + 'Tunnel_20180313-14/0mm/20180313_080112_16276736-0000/ant0_wholeleg.h5',
#                   base_folder + 'Tunnel_20180313-14/0mm/20180313_080521_16276736-0000/ant1.h5',
#                   base_folder + 'Tunnel_20180313-14/1mm/20180313_080514_16276712-0000/ant5.h5',
#                   base_folder + 'Tunnel_20180313-14/1mm/20180313_081138_16276712-0000/ant0.h5',
#                   base_folder + 'Tunnel_20180313-14/3mm/20180313_095224_16276718-0000/ant0.h5']
tracked_files = [ \
#                   base_folder + 'Tunnel_20180313-14/0mm/20180313_080112_16276736-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/1mm/20180313_080514_16276712-0000/ant5_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/1mm/20180313_081138_16276712-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/1mm/20180313_083905_16276712-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/1mm/20180313_084112_16276712-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/3mm/20180313_101520_16276718-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/5mm/20180313_105855_16276735-0000/ant0_wholeleg.h5']
subsample = 1

alltframes = np.empty((0,1,200,200))
for tfile in tracked_files:
    hf = h5py.File(tfile,'r')
    box_info = hf.get('box').value
    
    mfile = tfile[:-2]+'labels.mat'
    labeled_frames = loadmat(mfile, variable_names = 'positions')
    if sum(np.logical_not(np.isnan(labeled_frames['positions'][0,0,:]))) > 200:
        alltframes = np.append(alltframes, box_info[::2,:,:], axis=0)
    else:
        alltframes = np.append(alltframes, box_info[::subsample,:,:], axis=0)
    hf.close()

# save as a training model 
savefolder = '/media/gravishlab/SeagateExpansionDrive/AntTrack/LEAP_training/'
savelocation = savefolder+ 'Full_Training_Set.h5'
hf = h5py.File(savelocation,'w')
hf.create_dataset('box', data=alltframes)
hf.close()
print('save combined .h5')

# combine labels mat files using matlab engine (can't import a table into python)
eng = matlab.engine.start_matlab()
eng.combine_skeleton_mat(tracked_files,savefolder,subsample)

print('combined all .h5 labels')

save combined .h5
combined all .h5 labels


# Combine .h5 files to make larger training set - ONLY CERTAIN SKELETON POINTS


In [14]:
# import tracked h5s to combine
base_folder = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'
# FILES FOR 180621_133300 --> FullTrainingSet_v1
# tracked_files = [ \
#                   base_folder + 'Tunnel_20180313-14/0mm/20180313_080112_16276736-0000/ant0_wholeleg.h5',
#                   base_folder + 'Tunnel_20180313-14/0mm/20180313_080521_16276736-0000/ant1.h5',
#                   base_folder + 'Tunnel_20180313-14/1mm/20180313_080514_16276712-0000/ant5.h5',
#                   base_folder + 'Tunnel_20180313-14/1mm/20180313_081138_16276712-0000/ant0.h5',
#                   base_folder + 'Tunnel_20180313-14/3mm/20180313_095224_16276718-0000/ant0.h5']
tracked_files = [ \
                  base_folder + 'Tunnel_20180313-14/0mm/20180313_080112_16276736-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/1mm/20180313_080514_16276712-0000/ant5_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/1mm/20180313_081138_16276712-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/1mm/20180313_083905_16276712-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/1mm/20180313_083651_16276712-0000/ant0_onlyfeet.h5',
                  base_folder + 'Tunnel_20180313-14/1mm/20180313_084112_16276712-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/3mm/20180313_101520_16276718-0000/ant0_wholeleg.h5',
                  base_folder + 'Tunnel_20180313-14/3mm/20180313_095428_16276718-0000/ant0_onlyfeet.h5',
                  base_folder + 'Tunnel_20180313-14/5mm/20180313_111526_16276735-0000/ant0_onlyfeet.h5',
                  base_folder + 'Tunnel_20180313-14/5mm/20180313_112651_16276735-0000/ant0_onlyfeet.h5',
                  base_folder + 'Tunnel_20180313-14/5mm/20180313_105855_16276735-0000/ant0_wholeleg.h5']
subsample = 1

alltframes = np.empty((0,1,200,200))
for tfile in tracked_files:
    hf = h5py.File(tfile,'r')
    box_info = hf.get('box').value
    
    mfile = tfile[:-2]+'labels.mat'
    labeled_frames = loadmat(mfile, variable_names = 'positions')
    if sum(np.logical_not(np.isnan(labeled_frames['positions'][0,0,:]))) > 200:
        alltframes = np.append(alltframes, box_info[::2,:,:], axis=0)
        print('sampled half')
    else:
        alltframes = np.append(alltframes, box_info[::subsample,:,:], axis=0)
    hf.close()
    print(alltframes.shape)

# save as a training model 
savefolder = '/media/gravishlab/SeagateExpansionDrive/AntTrack/LEAP_training/'
savelocation = savefolder+ 'Full_Training_Set.h5'
hf = h5py.File(savelocation,'w')
hf.create_dataset('box', data=alltframes)
hf.close()
print('save combined .h5')


# combine labels mat files using matlab engine (can't import a table into python)
eng = matlab.engine.start_matlab()
eng.combine_skeleton_mat_partialskel(tracked_files,savefolder,subsample)

print('combined all .h5 labels')

sampled half
(192, 1, 200, 200)
(420, 1, 200, 200)
(1060, 1, 200, 200)
(1703, 1, 200, 200)
(2135, 1, 200, 200)
(2609, 1, 200, 200)
(3174, 1, 200, 200)
(3335, 1, 200, 200)
(3912, 1, 200, 200)
(4592, 1, 200, 200)
(5061, 1, 200, 200)
save combined .h5
combined all .h5 labels


# Plot leg and antennae excursions

## Get rid of untrustworthy points

In [ ]:
base_folder = '/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/'
predicted_files = glob.glob(os.path.join(vid_locations, 'Tunnel_20180313-14/**/*predictions.h5'))
fileOI = predicted_files[3]
print(fileOI)


In [6]:
# plot leg excursions

ant_num = 0
all_frames = [video.objects[obj]['frames'] for obj in video.objects]

pfunc_path = '/home/gravishlab/Documents/MATLAB/leap/leap/predict_box.py'
h5_path = "%s/ant%i.h5"%(savelocation, ant_num)
model_path = '/home/gravishlab/Documents/MATLAB/leap/models/fast_train/180614_094549-n=209/final_model.h5'
save_path = "%s/ant%i_predictions.h5"%(savelocation, ant_num)

# hf_b = h5py.File(h5_path,'r')
# box = hf_b.get('box')
hf = h5py.File(save_path,'r')
joint_loc = hf['positions_pred'][()].astype(np.float32) #hf.get('positions_pred')
joint_conf = hf['conf_pred'][()] #hf.get('conf_pred')
hf.close
joint_bad = joint_conf<0.4
joint_bad_idcs = np.swapaxes(np.tile(joint_bad,(2,1,1)), 0,1)
joint_loc[joint_bad_idcs] = np.nan
joint_loc = joint_loc-100



plt.figure()

plt.plot(all_frames[ant_num], joint_loc[:,0,7],'-') # R forelimb
plt.plot(all_frames[ant_num], joint_loc[:,0,11],'-') # R midlimb
plt.plot(all_frames[ant_num], joint_loc[:,0,15],'-') # R hindlimb

plt.plot(all_frames[ant_num], joint_loc[:,0,19],'-') # L forelimb
plt.plot(all_frames[ant_num], joint_loc[:,0,23],'-') # L midlimb
plt.plot(all_frames[ant_num], joint_loc[:,0,27],'-') # L hindlimb

plt.xlabel('frame')
plt.ylabel('travel direction (pix)')
plt.title('excursions of distal limb points\n%s'%(' -- ').join(h5_path.split('/')[-3:]))

Text(0.5,1,'excursions of distal limb points\n0mm -- 20180313_081350_16276736-0000 -- ant0.h5')

In [19]:
# plot antennae positions

plt.figure()
plt.plot(all_frames[ant_num], joint_loc[:,0,28],'-b') # L antenna tip
plt.plot(all_frames[ant_num], joint_loc[:,0,32],'-r') # R antenna tip
plt.xlabel('frame')
plt.ylabel('travel direction (pix)')
plt.title('antennae placement\n%s'%(' -- ').join(h5_path.split('/')[-3:]))

Text(0.5,1,'antennae placement\n0mm -- 20180313_081350_16276736-0000 -- ant0.h5')

In [162]:
%qtconsole